In [1]:
from sqlalchemy.ext.compiler import compiles
from sqlalchemy.sql.expression import FromClause

In [2]:
from sqlalchemy import (
        MetaData,
        create_engine,
        String,
        Integer,
        Table,
        Column,
    )
from sqlalchemy.sql import column
from sqlalchemy.orm import Session, mapper

In [10]:
# DEFINE VALUES CLAUSE
# https://github.com/sqlalchemy/sqlalchemy/wiki/PGValues

class values(FromClause):
    named_with_column = True

    def __init__(self, columns, *args, **kw):
        self._column_args = columns
        self.list = args
        self.alias_name = self.name = kw.pop("alias_name", None)

    def _populate_column_collection(self):
        for c in self._column_args:
            c._make_proxy(self)

    @property
    def _from_objects(self):
        return [self]


@compiles(values)
def compile_values(element, compiler, asfrom=False, **kw):
    columns = element.columns
    v = "VALUES %s" % ", ".join(
        "(%s)"
        % ", ".join(
            compiler.render_literal_value(elem, column.type)
            for elem, column in zip(tup, columns)
        )
        for tup in element.list
    )
    if asfrom:
        if element.alias_name:
            v = "(%s) AS %s (%s)" % (
                v,
                element.alias_name,
                (", ".join(c.name for c in element.columns)),
            )
        else:
            v = "(%s)" % v
    return v

In [3]:
m1 = MetaData()

class T(object):
    pass

t1 = Table(
    "mytable",
    m1,
    Column("mykey", Integer, primary_key=True),
    Column("mytext", String),
    Column("myint", Integer),
)
mapper(T, t1)

In [11]:
t2 = values(
    [
        column("mykey", Integer),
        column("mytext", String),
        column("myint", Integer),
    ],
    (1, "textA", 99),
    (2, "textB", 88),
    alias_name="myvalues",
)

In [12]:
e = create_engine("postgresql://alexbird:@localhost/synpuf1k", echo=True)
m1.create_all(e)

2020-06-02 10:36:46,970 INFO sqlalchemy.engine.base.Engine select version()
2020-06-02 10:36:46,971 INFO sqlalchemy.engine.base.Engine {}
2020-06-02 10:36:46,999 INFO sqlalchemy.engine.base.Engine select current_schema()
2020-06-02 10:36:47,000 INFO sqlalchemy.engine.base.Engine {}
2020-06-02 10:36:47,034 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-06-02 10:36:47,035 INFO sqlalchemy.engine.base.Engine {}
2020-06-02 10:36:47,038 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-06-02 10:36:47,039 INFO sqlalchemy.engine.base.Engine {}
2020-06-02 10:36:47,043 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2020-06-02 10:36:47,045 INFO sqlalchemy.engine.base.Engine {}
2020-06-02 10:36:47,048 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

In [13]:
sess = Session(e)

In [24]:
sess.query(T).filter(T.mykey == t2.c.mykey).all()

2020-06-02 10:38:56,358 INFO sqlalchemy.engine.base.Engine SELECT mytable.mykey AS mytable_mykey, mytable.mytext AS mytable_mytext, mytable.myint AS mytable_myint 
FROM mytable, (VALUES (1, 'textA', 99), (2, 'textB', 88)) AS myvalues (mykey, mytext, myint) 
WHERE mytable.mykey = myvalues.mykey
2020-06-02 10:38:56,359 INFO sqlalchemy.engine.base.Engine {}


[]

In [28]:
str(t1.join(t2, t1.c.mykey == t2.c.mykey))

"mytable JOIN (VALUES (1, 'textA', 99), (2, 'textB', 88)) AS myvalues (mykey, mytext, myint) ON mytable.mykey = myvalues.mykey"

In [3]:
import plotly.graph_objects as go
import plotly.express as px

In [5]:
px.colors.qualitative.T10[0]

'#4C78A8'

In [22]:
print("\n".join(str(sess.query(T).filter(T.mykey == t2.c.mykey)).split(",")))

SELECT mytable.mykey AS mytable_mykey
 mytable.mytext AS mytable_mytext
 mytable.myint AS mytable_myint 
FROM mytable
 (VALUES (1
 'textA'
 99)
 (2
 'textB'
 88)) AS myvalues (mykey
 mytext
 myint) 
WHERE mytable.mykey = myvalues.mykey
